In [1]:
from EpubProcessor import EPUBProcessor
from EpubSummarizer import EPUBSummarizer

In [2]:
processor = EPUBProcessor("./books/nexus.epub")
processor.A_load_epub()
p = processor.B_parse_chapters()
processor.display_parsed_chapter()

/home/teo/Documents/EpubParser/env_pub/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/home/teo/Documents/EpubParser/env_pub/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):



[0] (paragraph) -> <p class="acmh-ac-pg"><a id="_idTextAnchor002"></a>BY YUVAL NOAH HARARI</p>
[1] (paragraph) -> <p class="disclaimer"><a id="_idTextAnchor004"></a>This is an uncorrected ebook file. Please do not quote for publication until you check your copy against the finished book.</p>
[2] (paragraph) -> <p class="dedf-ded-pg"><a id="_idTextAnchor005"></a>To Itzik with love, and to all who love wisdom.</p>
[3] (paragraph) -> <p class="fmt-toc-pg">Contents</p>
[4] (paragraph) -> <p class="fmt-pro-pg"><a id="_idTextAnchor006"></a>Prologue</p>
[5] (paragraph) -> <p class="pn-part-pg" id="_idParaDest-1"><a id="_idTextAnchor007"></a>PART I</p>
[6] (paragraph) -> <p class="cn-chap-pg" id="_idParaDest-2"><a id="_idTextAnchor008"></a>CHAPTER 1</p>
[7] (paragraph) -> <p class="cn-chap-pg" id="_idParaDest-3"><a id="_idTextAnchor009"></a>CHAPTER 2</p>
[8] (paragraph) -> <p class="cn-chap-pg" id="_idParaDest-4"><a id="_idTextAnchor010"></a>CHAPTER 3</p>
[9] (paragraph) -> <p class="cn-chap-

In [3]:
classes = []
for cI, c in enumerate(processor.parsed_chapters):
        for sI, s in enumerate(c):
            classes.extend(s[2])

print(set(classes))

{'tocbm-alt', 'p', 'pt', 'paft', 'fmt-toc-pg', 'art-text', 'pcon', 'fmh1', 'extl', 'tocct', 'ul', 'dedf-ded-pg', '_idGenObjectAttribute-4', 'fmp', 'h1', 'ulf', 'exts', 'ornfmbm', '_idGenObjectAttribute-6', 'orn2', 'illcapf', 'pn-part-pg', 'extf', 'acl', 'tocpt', 'crtf', 'fmpf', 'ct', 'tocfm', 'disclaimer', '_idGenObjectAttribute-9', 'bmt-ack-pg', 'crtf-crt-pg', 'en-alt1', '_idGenObjectAttribute-5', '_idGenObjectAttribute-7', 'ct-epl-pg', 'h1-alt', '_idGenObjectAttribute-11', 'ull', 'ataf', 'en-alt3', 'acmh-ac-pg', 'cn-chap-pg', 'tocbm', 'atat-ata-pg', '_idGenObjectAttribute-8', 'en', 'bmpf', '_idGenObjectAttribute-10', 'orn3', 'bmh1', 'fmt-pro-pg', 'ded', 'crt', 'bmt-nt-pg', 'pf', 'aclf', 'fmpaft'}


In [4]:
titles_tags = ['cn-chap-pg', 'pn-part-pg', 'ct', 'fmt-pro-pg', 'ct-epl-pg', 'h1', 'h2', 'h3', 'h4']

def set_section_types_by_classes_and_clean_text(self):
    for cI, c in enumerate(self.parsed_chapters):
        for sI, s in enumerate(c):
            classes_tags = s[2]

            if any(tag in classes_tags for tag in titles_tags):
                typ = 'title'
                text = self.parsed_chapters[cI][sI][1].get_text()
                self.parsed_chapters[cI][sI] = (typ, text)
            else:
                self.parsed_chapters[cI][sI] = (s[0], s[1].get_text())

In [5]:
from types import MethodType
processor.C_function__set_section_types_by_classes_and_clean_text = MethodType(set_section_types_by_classes_and_clean_text, processor)

processor.D_set_section_types_by_classes_and_clean_text()

for c in processor.parsed_chapters:
    print(c[0])

organized_book = processor.E_organize_book()

('paragraph', 'BY YUVAL NOAH HARARI')
('paragraph', 'This is an uncorrected ebook file. Please do not quote for publication until you check your copy against the finished book.')
('paragraph', 'To Itzik with love, and to all who love wisdom.')
('paragraph', 'Contents')
('title', 'Prologue')
('title', 'PART I')
('title', 'CHAPTER 1')
('title', 'CHAPTER 2')
('title', 'CHAPTER 3')
('title', 'CHAPTER 4')
('title', 'CHAPTER 5')
('title', 'PART II')
('title', 'CHAPTER 6')
('title', 'CHAPTER 7')
('title', 'CHAPTER 8')
('title', 'PART III')
('title', 'CHAPTER 9')
('title', 'CHAPTER 10')
('title', 'CHAPTER 11')
('title', 'Epilogue')
('paragraph', 'Acknowledgments')
('paragraph', 'Notes')
('paragraph', 'ABOUT THE AUTHOR')


In [6]:
organized_book['PART I'] = [('title', organized_book['PART I'][1][0] + ' - ' + organized_book['PART I'][1][-1])]
organized_book['PART II'] = [('title', organized_book['PART II'][1][0] + ' - ' + organized_book['PART II'][1][-1])]
organized_book['PART III'] = [('title', organized_book['PART III'][1][0] + ' - ' + organized_book['PART III'][1][-1])]

In [7]:
summarizer = EPUBSummarizer("Nexus by Yuval Noah Harari", organized_book, openai_key)


In [8]:

SYSTEM_INSTRUCTION = "You are a helpful assistant that summarizes paragraphs from books. Return markdown formatting without any tags around it nor mentions of the title of the section."
def function_summarize_section(text):
    return [
        {"role": "system", "content": SYSTEM_INSTRUCTION },
        {"role": "user", "content": f"Summarize the following text: {text}"}
    ]

keys_to_skip = ['','INDEX', "PART I", "PART II", "PART III"]
#keys_to_skip = ['', 'INDEX', "PART II", "PART III", 'CHAPTER 1', 'CHAPTER 2', 'CHAPTER 3', 'CHAPTER 4', 'CHAPTER 5', 'CHAPTER 6', 'CHAPTER 7', 'CHAPTER 8', 'CHAPTER 9', 'CHAPTER 10', 'CHAPTER 11', 'Epilogue', 'Notes']

summarizer.A_summarize_sections_for_chapters(function_summarize_section, keys_to_skip)

Skipping: ['', 'INDEX', 'PART I', 'PART II', 'PART III']

Prologue
	10/77
	20/77
	30/77
	40/77
	50/77
	60/77
	70/77
CHAPTER 1
	10/49
	20/49
	30/49
	40/49
CHAPTER 2
	10/72
	20/72
	30/72
	40/72
	50/72
	60/72
	70/72
CHAPTER 3
	10/97
	20/97
	30/97
	40/97
	50/97
	60/97
	70/97
	80/97
	90/97
CHAPTER 4
	10/151
	20/151
	30/151
	40/151
	50/151
	60/151
	70/151
	80/151
	90/151
	100/151
	110/151
	120/151
	130/151
	140/151
	150/151
CHAPTER 5
	10/236
	20/236
	30/236
	40/236
	50/236
	60/236
	70/236
	80/236
	90/236
	100/236
	110/236
	120/236
	130/236
	140/236
	150/236
	160/236
	170/236
	180/236
	190/236
	200/236
	210/236
	220/236
	230/236
CHAPTER 6
	10/113
	20/113
	30/113
	40/113
	50/113
	60/113
	70/113
	80/113
	90/113
	100/113
	110/113
CHAPTER 7
	10/76
	20/76
	30/76
	40/76
	50/76
	60/76
	70/76
CHAPTER 8
	10/146
	20/146
	30/146
	40/146
	50/146
	60/146
	70/146
	80/146
	90/146
	100/146
	110/146
	120/146
	130/146
	140/146
CHAPTER 9
	10/133
	20/133
	30/133
	40/133
	50/133
	60/133
	70/133
	80/133
	90/133
	1

In [10]:
summarizer.B_summarize_chapter_from_sections_summaries()

Prologue
CHAPTER 1
CHAPTER 2
CHAPTER 3
CHAPTER 4
CHAPTER 5
CHAPTER 6
CHAPTER 7
CHAPTER 8
CHAPTER 9
CHAPTER 10
CHAPTER 11
Epilogue


In [11]:
summarizer.C_summarize_book_from_chapters()

Prologue
CHAPTER 1
CHAPTER 2
CHAPTER 3
CHAPTER 4
CHAPTER 5
CHAPTER 6
CHAPTER 7
CHAPTER 8
CHAPTER 9
CHAPTER 10
CHAPTER 11
Epilogue


In [12]:
summarizer.D_summarize_chapter_from_sections_summaries_and_book_summary()

Prologue
CHAPTER 1
CHAPTER 2
CHAPTER 3
CHAPTER 4
CHAPTER 5
CHAPTER 6
CHAPTER 7
CHAPTER 8
CHAPTER 9
CHAPTER 10
CHAPTER 11
Epilogue


In [13]:
summarizer.E_summarized_sections_with_book_and_chapter_context.keys()

dict_keys([])

In [14]:
summarizer.E_summarize_sections_with_book_and_chapter_summaries()

Prologue
	Prologue
CHAPTER 1
	CHAPTER 1
	What Is Information?
	WHAT IS TRUTH?
	WHAT INFORMATION DOES
	INFORMATION IN HUMAN HISTORY
CHAPTER 2
	CHAPTER 2
	Stories: Unlimited Connections
	INTERSUBJECTIVE ENTITIES
	THE POWER OF STORIES
	THE NOBLE LIE
	THE PERENNIAL DILEMMA
CHAPTER 3
	CHAPTER 3
	Documents: The Bite of the Paper Tigers
	TO KILL A LOAN
	BUREAUCRACY
	BUREAUCRACY AND THE SEARCH FOR TRUTH
	THE DEEP STATE
	THE BIOLOGICAL DRAMAS
	LET’S KILL ALL THE LAWYERS
	THE MIRACLE DOCUMENT
CHAPTER 4
	CHAPTER 4
	Errors: The Fantasy of Infallibility
	TAKING HUMANS OUT OF THE LOOP
	THE INFALLIBLE TECHNOLOGY
	THE MAKING OF THE HEBREW BIBLE
	THE INSTITUTION STRIKES BACK
	THE SPLIT BIBLE
	THE ECHO CHAMBER
	PRINT, SCIENCE, AND WITCHES
	THE SPANISH INQUISITION TO THE RESCUE
	THE DISCOVERY OF IGNORANCE
	SELF-CORRECTING MECHANISMS
	THE DSM AND THE BIBLE
	PUBLISH OR PERISH
	THE LIMITS OF SELF-CORRECTION
CHAPTER 5
	CHAPTER 5
	Decisions: A Brief History of Democracy and Totalitarianism
	MAJORITY DICTATORS

In [19]:
summarized = summarizer.E_summarized_sections_with_book_and_chapter_context
summarized.keys()

dict_keys(['Prologue', 'CHAPTER 1', 'CHAPTER 2', 'CHAPTER 3', 'CHAPTER 4', 'CHAPTER 5', 'CHAPTER 6', 'CHAPTER 7', 'CHAPTER 8', 'CHAPTER 9', 'CHAPTER 10', 'CHAPTER 11', 'Epilogue'])

In [25]:
from openai import OpenAI
import requests

def summarize_text_OPENAI(messages, openai_key, model):
    client = OpenAI(api_key=openai_key)

    response = client.chat.completions.create(
        model=model, 
        messages = messages,
    )
    summary = response.choices[0].message.content.strip()
    return summary, messages

def summarize_text_LMSTUDIO(messages, tries=5):
    
    payload = {
        "model": "phi-4@q6_k", 
        "messages": messages,
    }
    
    

    while tries > 0:
        try:
            response = requests.post(
                "http://localhost:24236/v1/chat/completions",
                headers={"Content-Type": "application/json"},
                json=payload,
                timeout=15
            )
            if response.status_code == 200:
                result = response.json()
                summary = result.get("choices", [{}])[0].get("message", {}).get("content", "").strip()
                return summary, messages
            else:
                tries -=1 
        except:
            tries -=1

    raise Exception(f"Tries ran out --> {messages}")


SYSTEM_INSTRUCTION = "You are a helpful assistant that summarizes paragraphs from books. Return markdown formatting without any tags around it."

def summarize_text(messages, openai_key=None, model="gpt-4o-mini"):
    if openai_key is not None:
        return summarize_text_OPENAI(messages, openai_key, model)
        
    return summarize_text_LMSTUDIO(messages)


In [29]:
from HTMLGenerator import *

part_summaries = {
    'PART I': ['CHAPTER 1', 'CHAPTER 2', 'CHAPTER 3', 'CHAPTER 4', 'CHAPTER 5'],
    'PART II': [ 'CHAPTER 6', 'CHAPTER 7', 'CHAPTER 8'],
    'PART III': ['CHAPTER 9', 'CHAPTER 10', 'CHAPTER 11'],
}

def F_summarize_by_selection(selection_dict):
    result = {}
    for k, chapter_keys in selection_dict.items():
        concatenated = ""
        for title in chapter_keys:
            chap_summary = summarizer.E_summarized_sections_with_book_and_chapter_context[title]['summary']['text']
            print(title)
            concatenated += f"## Chapter '{title}'" + "\n"
            concatenated += f"{chap_summary}"
            concatenated += "\n\n"

        instruction = "Summarize the following text composed of summaries of chapters of a book."
        instruction += f"\nThis context comes from the book {summarizer.book_title} with its summary:\n {summarizer.C_book_summary}"
        instruction += f"Create a summary for this group of chapters named '{k}' with the contents of the summaries being: {concatenated}"
        messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes paragraphs from books. Use paragraphs to break logically and to enable better structure. Return markdown formatting without any tags around it."},
                    {"role": "user", "content": instruction}
            ]

        part_summary, messages = summarize_text(messages=messages, openai_key=summarizer.openai_key, model="gpt-4o")
        result[k] = part_summary
    return result

part_summaries = F_summarize_by_selection(part_summaries)

CHAPTER 1
CHAPTER 2
CHAPTER 3
CHAPTER 4
CHAPTER 5
CHAPTER 6
CHAPTER 7
CHAPTER 8
CHAPTER 9
CHAPTER 10
CHAPTER 11


In [40]:
import re
import markdown
from bs4 import BeautifulSoup

def text_to_paragraphs(text):
    """Convert text with newlines to HTML paragraphs, handling both plain text and markdown.
    
    Args:
        text (str): Input text that may contain markdown or plain text
        
    Returns:
        str: HTML formatted text with proper paragraph tags
    """
    # First, check if the text appears to contain markdown
    markdown_indicators = ['#', '*', '_', '`', '[', '>', '-', '+']
    has_markdown = any(indicator in text for indicator in markdown_indicators)
    
    if has_markdown:
        # Convert markdown to HTML using the markdown library
        md = markdown.Markdown(extensions=['extra'])
        html_content = md.convert(text)
        
        # Clean up the HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Handle standalone text nodes by wrapping them in paragraphs
        for text_node in soup.find_all(text=True):
            if text_node.parent.name == 'body' or text_node.parent.name == '[document]':
                # Only wrap if it's not just whitespace
                if text_node.strip():
                    new_p = soup.new_tag('p')
                    text_node.wrap(new_p)
        
        return str(soup)
    else:
        # Handle plain text by splitting on newlines and wrapping in paragraph tags
        paragraphs = text.split('\n')
        return '\n'.join([f"<p>{p.strip()}</p>" for p in paragraphs if p.strip()])

def generate_html(book_title, book_summary, book_chapters, part_summaries={}):
    """Generate HTML content from book dictionary with markdown support.
    
    Args:
        book_dict (dict): Dictionary containing book content and structure
        
    Returns:
        str: Complete HTML document as string
    """
    # HTML template with styles
    html_template = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <style>
        @page {
            size: A4;
            margin: 2.5cm;
        }
        body {
            font-family: "Times New Roman", Times, serif;
            font-size: 12pt;
            line-height: 1.5;
            max-width: 21cm;
            margin: 0 auto;
            padding: 20px;
        }
        h1 {
            font-size: 24pt;
            text-align: center;
            margin-top: 40px;
            margin-bottom: 20px;
        }
        h2 {
            font-size: 18pt;
            margin-top: 30px;
        }
        .section-container {
            margin-bottom: 20px;
        }
        h3 {
            font-size: 14pt;
            margin-top: 20px;
            margin-bottom: 10px;
        }
        
        h4 {
            font-size: 13pt;
            margin-top: 20px;
            margin-bottom: 10px;
        }
        p {
            margin-bottom: 12pt;
            text-align: justify;
            orphans: 3;
            widows: 3;
            word-wrap: break-word;
            overflow-wrap: break-word;
            hyphens: auto;
            -webkit-hyphens: auto;
            -ms-hyphens: auto;
            page-break-inside: avoid;
        }
        .summary {
            margin: 20px 0;
            font-style: italic;
        }
        .chapter-summary {
            margin: 18px 0;
            font-style: italic;
            padding: 12px;
            background-color: #f8f8f8;
            border-left: 3px solid #333;
        }
        .page-break {
            page-break-after: always;
        }
        .chapter-container {
            margin-bottom: 40px;
        }
        .title-container {
            page-break-inside: avoid;
            margin-bottom: 30px;
        }
        hr {
            margin: 20px 0;
            border: none;
            border-top: 1px solid #000;
        }
        /* Additional styles for markdown elements */
        blockquote {
            margin: 15px 30px;
            padding-left: 15px;
            border-left: 3px solid #ccc;
        }
        code {
            font-family: monospace;
            background-color: #f5f5f5;
            padding: 2px 4px;
            border-radius: 3px;
        }
        pre {
            background-color: #f5f5f5;
            padding: 15px;
            border-radius: 5px;
            overflow-x: auto;
        }
        em {
            font-style: italic;
        }
        strong {
            font-weight: bold;
        }
    </style>
</head>
<body>
"""
    
    # Add title and main summary
    html = html_template
    html += "<div class='title-container'>\n"
    html += f"<h1>{book_title}</h1>\n"
    html += f"<div class='summary'>{text_to_paragraphs(book_summary)}</div>\n"
    html += "</div>\n"
    html += "<hr>\n"

    html += "<div class='page-break'></div>"

    if part_summaries is None:
        # Add chapter summaries overview
        for chapter_title, chapter in book_chapters.items():
            html += f"<div class='chapter-summary'><h3>{chapter_title}</h3>\n"
            html += f"{text_to_paragraphs(chapter['summary']['text'])}</div>\n"
    else:
        for part_title, part_summary in part_summaries.items():
            html += f"<div class='chapter-summary'><h3>{part_title}</h3>\n"
            html += f"{text_to_paragraphs(part_summary)}</div>\n"
    
    html += "<div class='page-break'></div>"

    # Process each chapter in detail
    for chapter_title, chapter in book_chapters.items():
        html += "<div class='chapter-container'>\n"
        html += f"<div class='chapter-summary'><h2>{chapter_title}</h2>\n"
        if part_summaries is not None:
            html += f"{text_to_paragraphs(chapter['summary']['text'])}</div>\n"
        else:
            html += "</div>\n"

        html += "<hr>\n"
        
        # Process each section
        for section in chapter['content']:
            html += "<div class='section-container'>\n"
            html += f"<h3>{section['title']}</h3>\n"
            html += text_to_paragraphs(section['summary']['text']) + "\n"
            html += "</div>\n"

        html += "<div class='page-break'></div>"
            
        html += "</div>\n"
    
    html += "</body></html>"
    return html

def save_html(html_content, filename="book_summary.html"):
    """Save the generated HTML to a file
    
    Args:
        html_content (str): HTML content to save
        filename (str): Output filename
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)

In [41]:
html = generate_html(summarizer.book_title, summarizer.C_book_summary, summarizer.E_summarized_sections_with_book_and_chapter_context, part_summaries)
save_html(html, filename=summarizer.get_fingerprint() + ".html")

/tmp/ipykernel_34559/3718021284.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for text_node in soup.find_all(text=True):


In [31]:
part_summaries

{'PART I': '**PART I Summary of "Nexus" by Yuval Noah Harari**\n\n## Chapter 1\nIn the first chapter, Harari explores the multifaceted concept of information, highlighting its historical role in shaping societies. He argues that while information traditionally serves to represent reality, its primary function might be to foster connectivity among humans. This perspective is supported by historical examples, such as the use of astrology and music as tools for human connection, rather than truthful representation. The chapter underscores information\'s power to craft collective beliefs and societal narratives that transcend mere factual accuracy.\n\n## Chapter 2\nHarari delves deeper into the unique human capability for extensive cooperation through storytelling. By crafting shared narratives, humans built intersubjective realities like religions and nations, enabling cooperation beyond personal bounds and transforming societal dynamics. These stories foster collective memory and identit